In [50]:
# !pip install numpy
# !pip install numpy-stl
# !pip install tripy
# !pip install plotly

In [51]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from stl import mesh
# from google.colab import drive
import numpy as np

In [52]:


# drive.mount('/content/drive')
# drive_path = "/content/drive/MyDrive/Research/PhD/Scripts/"
# stl_file_path = drive_path + "test_part.stl"
# stl_file_path = "./test_part.stl"
# stl_file_path = "./vessel_half.stl"
# stl_file_path = "./circle_split.stl"
# stl_file_path = "./vessel_quarter.stl"
# stl_file_path = "./simple_split.stl"
stl_file_path = "./neuro_model.stl"




mesh_data = mesh.Mesh.from_file(stl_file_path)

# Extract vertices
vertices = mesh_data.vectors.reshape((-1, 3))

# Create a 3D scatter plot
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

fig.add_trace(go.Scatter3d(
    x=vertices[:, 0],
    y=vertices[:, 1],
    z=vertices[:, 2],
    mode='markers',
    marker=dict(size=2),
))

# Set layout
fig.update_layout(scene=dict(aspectmode="data"))

# Show the plot
fig.show()

In [53]:
def on_top_surface(triangle, top_height):
  # print(triangle[:,2])
  return np.all(triangle[:,2]>top_height)

In [54]:
print(vertices.shape)
np.set_printoptions(suppress=True, precision=2)

height_tolerance = 1 #mm

# print(vertices)
# n_scores[:, 1:-1].max(axis=1)
# print("max z height", str(vertices[:,2].max()))

height_lb = vertices[:,2].max()-height_tolerance

triangles =  np.array([vertices[0:3,:]])

# organizes vertices into triangles
for i in range(int(vertices.shape[0]/3)):
  temp_tri = np.array([vertices[3*i:3*(i+1),:]])
  triangles = np.vstack((triangles, temp_tri))


# extracts top surface triangles
surface_tris = np.array([])
for ind in range(triangles.shape[0]):
  # checks if on top surface
  if(on_top_surface(triangles[ind], height_lb)):
    if(surface_tris.size == 0):
      surface_tris = np.array([triangles[ind]])
      print("surface triangle start shape", str(surface_tris.shape))
    else:
      surface_tris = np.vstack((surface_tris, [triangles[ind]]))

print(surface_tris)


(15840, 3)
surface triangle start shape (1, 3, 3)
[[[298.88 856.26   4.  ]
  [298.57 846.74   4.  ]
  [300.04 852.37   4.  ]]

 [[300.04 852.37   4.  ]
  [298.57 846.74   4.  ]
  [300.95 847.96   4.  ]]

 [[300.95 847.96   4.  ]
  [298.57 846.74   4.  ]
  [298.49 828.85   4.  ]]

 ...

 [[279.78 826.33   4.  ]
  [279.57 817.36   4.  ]
  [280.05 815.43   4.  ]]

 [[279.78 826.33   4.  ]
  [280.05 815.43   4.  ]
  [280.77 813.71   4.  ]]

 [[279.35 822.78   4.  ]
  [279.32 819.48   4.  ]
  [279.57 817.36   4.  ]]]


In [55]:
def shares_side(poly1,poly2):
  shared_vertices = 0
  # print("before squeeze")
  # print(poly1.shape)
  # print(poly2.shape)
  if(len(poly2.shape)>2):
    poly2 = np.squeeze(poly2)
  if(len(poly1.shape)>2):
    poly1 = np.squeeze(poly1)
  # print("after squeeze")
  # print(poly1.shape)
  # print(poly2.shape)
  # print("poly 1 shape", str(poly1.shape), "poly2 shape", str(poly2.shape))
  for i in range(poly1.shape[0]):
    for j in range(poly2.shape[0]):
      # print(str(poly1[i]), " =?", str(poly2[j]))
      # print(np.all(poly1[i] ==poly2[j]))
      # if(np.all(poly1[i] ==poly2[j])):
      if(np.allclose(poly1[i], poly2[j],atol=0.01)):
        
        shared_vertices = shared_vertices+1
        if(shared_vertices >= 2):
          return True
  # if(shared_vertices>0):
    # print(str(shared_vertices), "shared vertices")
  return False

def shares_point(poly1,poly2):
  # print("before squeeze")
  # print(poly1.shape)
  # print(poly2.shape)
  if(len(poly2.shape)>2):
    poly2 = np.squeeze(poly2)
  if(len(poly1.shape)>2):
    poly1 = np.squeeze(poly1)
  # print("after squeeze")
  # print(poly1.shape)
  # print(poly2.shape)
  # print("poly 1 shape", str(poly1.shape), "poly2 shape", str(poly2.shape))
  for i in range(poly1.shape[0]):
    for j in range(poly2.shape[0]):
      # print(str(poly1[i]), " =?", str(poly2[j]))
      # print(np.all(poly1[i] ==poly2[j]))
      # if(np.all(poly1[i] ==poly2[j])):
      if(np.allclose(poly1[i], poly2[j],atol=0.01)):
          return True
  # if(shared_vertices>0):
    # print(str(shared_vertices), "shared vertices")
  return False

def combine(poly1,poly2):
    poly2 = np.squeeze(poly2)
    poly1 = np.squeeze(poly1)
    combined_poly = np.vstack((poly1,poly2))
    # print(combined_poly)
    return combined_poly


def combine_polygons(triangles):
    combined_polygons = [triangles[0]]
    # print(len(combined_polygons))
    # iterates through triangles
    for tri_ind in range(1,triangles.shape[0]):
      added_to_existing = False
      # checks combined polygon list
      for poly_ind in range(len(combined_polygons)):
        # checks for shared side
        # print("Checking ", str(poly_ind), "of ", str(len(combined_polygons)))
        # print("comparing ", str(triangles[tri_ind]), "to ", str(combined_polygons[poly_ind]))
        if(shares_side(triangles[tri_ind], combined_polygons[poly_ind])):
          combined_polygons[poly_ind] = combine(triangles[tri_ind], combined_polygons[poly_ind])
          added_to_existing = True
          # print("added  " , str(tri_ind), " to existing at", str(poly_ind))
          break
      if not added_to_existing:
          # combined_polygons = np.vstack(combined_polygons, triangles[tri_ind])

          combined_polygons.append(np.array([triangles[tri_ind]]))
          # print("added  " , str(tri_ind), " as new polygon")
          # print("Appended polygons")
          # print(combined_polygons)

      # print(combined_polygons)

    # print(len(combined_polygons))
    # print(combined_polygons)
    # print("----")
    return combined_polygons


def remove_repeat_from_polys(polygons):
  # print(polygons)
  new_polygons = []

  for poly in polygons:
    # print(poly.shape)
    new_poly = poly[0]
    # print("New poly", str(new_poly))
    # print("---")
    for row in range(1,poly.shape[0]):
      if not shares_point(new_poly, np.array([poly[row,:]])):
        new_poly = combine(new_poly, poly[row,:])
        # print("New Side added!")
        # print(new_poly)
      else:
        # print("Repeat side not added")
        # print(new_poly)
         pass
    # print("updated polygon:" )
    # print(new_poly)
    new_polygons.append(new_poly)

  return new_polygons




# print(surface_tris)
polygons = combine_polygons(surface_tris)
print("Before removal")
for poly in polygons:
  print(poly)
  print('---')

polygons = remove_repeat_from_polys(polygons)
print("after removal")

for poly in polygons:
  print(poly)
  print('---')

# print(surface_tris)


Before removal
[[298.44 809.41   4.  ]
 [298.12 819.23   4.  ]
 [297.37 809.19   4.  ]
 [300.69 810.16   4.  ]
 [298.12 819.23   4.  ]
 [298.44 809.41   4.  ]
 [298.49 828.85   4.  ]
 [298.12 819.23   4.  ]
 [300.69 810.16   4.  ]
 [303.54 815.88   4.  ]
 [303.17 812.3    4.  ]
 [303.57 813.76   4.  ]
 [303.13 818.43   4.  ]
 [303.17 812.3    4.  ]
 [303.54 815.88   4.  ]
 [303.13 818.43   4.  ]
 [302.25 811.13   4.  ]
 [303.17 812.3    4.  ]
 [301.94 825.29   4.  ]
 [302.25 811.13   4.  ]
 [303.13 818.43   4.  ]
 [301.94 825.29   4.  ]
 [300.69 810.16   4.  ]
 [302.25 811.13   4.  ]
 [301.64 829.31   4.  ]
 [300.69 810.16   4.  ]
 [301.94 825.29   4.  ]
 [301.64 829.31   4.  ]
 [298.49 828.85   4.  ]
 [300.69 810.16   4.  ]
 [301.59 833.32   4.  ]
 [298.49 828.85   4.  ]
 [301.64 829.31   4.  ]
 [301.64 838.31   4.  ]
 [298.49 828.85   4.  ]
 [301.59 833.32   4.  ]
 [301.51 843.12   4.  ]
 [298.49 828.85   4.  ]
 [301.64 838.31   4.  ]
 [300.95 847.96   4.  ]
 [298.49 828.85   4.  ]
 

In [56]:
# https://stackoverflow.com/questions/6989100/sort-points-in-clockwise-order
def less(pt1, pt2, center):
    # print("P1: ", str(pt1))
    # print("P2: ", str(pt2))
    # print("center: ", str(center))
    if (pt1[0] - center[0] >= 0 and pt2[0] - center[0] < 0):
        return True
    if (pt1[0] - center[0] < 0 and pt2[0] - center[0] >= 0):
        return False
    if (pt1[0] - center[0] == 0 and pt2[0] - center[0] == 0): 
        if (pt1[1] - center[1] >= 0 or pt2[1] - center[1] >= 0):
            return pt1[1] > pt2[1]
        return pt2[1] > pt1[1]

    # // compute the cross product of vectors (center -> a) x (center -> b)
    det = (pt1[0] - center[0]) * (pt2[1] - center[1]) - (pt2[0] - center[0]) * (pt1[1] - center[1])
    if (det < 0):
        return True
    if (det > 0):
        return False

    # // points a and b are on the same line from the center
    # // check which point is closer to the center
    d1 = (pt1[0] - center[0]) * (pt1[0] - center[0]) + (pt1[1] - center[1]) * (pt1[1] - center[1])
    d2 = (pt2[0] - center[0]) * (pt2[0] - center[0]) + (pt2[1] - center[1]) * (pt2[1] - center[1])
    return d1 > d2


#sort in clockwise order. bubble.
def sort_clockwise(polygon):
    new_poly = polygon
    
    center = np.mean(new_poly, axis= 0)
    print("center: ", str(center))
    n = new_poly.shape[0]
    
    # Traverse through all array elements
    for i in range(n):
        swapped = False
        
        # Last i elements are already in place
        for j in range(0, n-i-1):
            # Traverse the array from 0 to n-i-1
            # Swap if the element found is greater
            # than the next element
            if not less(new_poly[j,:],new_poly[j+1,:], center):
                # new_poly[j,:], new_poly[j+1,:] = new_poly[j+1,:], new_poly[j,:]
                new_poly[[j,j+1]] = new_poly[[j+1,j]]
                swapped = True
        if (swapped == False):
            break
    
    return new_poly


# print("before: ")
# print(polygons[0])
# print("after: ")
# print(sort_clockwise(polygons[0]))


for poly in polygons:
  print(poly)
  print (" becomes: ")
  print(sort_clockwise(poly))
  print('---')
  poly = sort_clockwise(poly)


    # # print("New poly", str(new_poly))
    # # print("---")
    # for row in range(n):
    #   if not shares_point(new_poly, np.array([poly[row,:]])):
    #     new_poly = combine(new_poly, poly[row,:])
    #     # print("New Side added!")
    #     # print(new_poly)
    #   else:
    #     # print("Repeat side not added")
    #     # print(new_poly)
    #      pass


[[298.44 809.41   4.  ]
 [298.12 819.23   4.  ]
 [297.37 809.19   4.  ]
 [300.69 810.16   4.  ]
 [298.49 828.85   4.  ]
 [303.54 815.88   4.  ]
 [303.17 812.3    4.  ]
 [303.57 813.76   4.  ]
 [303.13 818.43   4.  ]
 [302.25 811.13   4.  ]
 [301.94 825.29   4.  ]
 [301.64 829.31   4.  ]
 [301.59 833.32   4.  ]
 [301.64 838.31   4.  ]
 [301.51 843.12   4.  ]
 [300.95 847.96   4.  ]
 [298.57 846.74   4.  ]
 [300.04 852.37   4.  ]
 [298.88 856.26   4.  ]]
 becomes: 
center:  [300.82 827.42   4.  ]
[[300.95 847.96   4.  ]
 [301.51 843.12   4.  ]
 [301.64 838.31   4.  ]
 [301.59 833.32   4.  ]
 [301.64 829.31   4.  ]
 [301.94 825.29   4.  ]
 [303.13 818.43   4.  ]
 [303.54 815.88   4.  ]
 [303.57 813.76   4.  ]
 [303.17 812.3    4.  ]
 [302.25 811.13   4.  ]
 [300.69 810.16   4.  ]
 [298.44 809.41   4.  ]
 [297.37 809.19   4.  ]
 [298.12 819.23   4.  ]
 [298.49 828.85   4.  ]
 [298.57 846.74   4.  ]
 [298.88 856.26   4.  ]
 [300.04 852.37   4.  ]]
---
center:  [300.82 827.42   4.  ]
[[345.1

In [57]:
# moving everything to the first quadrant.

def reset_origin_tris(polygons, x_offset=0.0, y_offset=0.0):
	min_x =  float("inf")
	min_y =  float("inf")
	# getting minimum indices
	for tri_ind in range(int(len(polygons))):
		# if(surface_tris[tri_ind].shape[0] != 3):
		# 	print("Shape is not a triangle! ", str(surface_tris[tri_ind].shape[0]), " Points detected!")
		# 	print(surface_tris[tri_ind])
		for point_ind in range(polygons[tri_ind].shape[0]):		
			if polygons[tri_ind,point_ind,0] < min_x:
				min_x = polygons[tri_ind,point_ind,0] 
			if polygons[tri_ind,point_ind,1] < min_y:
				min_y = polygons[tri_ind,point_ind,1] 
	
	print("min x ", min_x)
	print("min y ", min_y)
	# updating all points
	for tri_ind in range(int(len(polygons))):

		for point_ind in range(polygons[tri_ind].shape[0]):
			if(min_x<0):
				polygons[tri_ind][point_ind,0] = polygons[tri_ind][point_ind,0]+abs(min_x)+x_offset
			else:
				polygons[tri_ind][point_ind,0] = polygons[tri_ind][point_ind,0]-abs(min_x)+x_offset

			if(min_y<0):
				polygons[tri_ind][point_ind,1] = polygons[tri_ind][point_ind,1]+abs(min_y)+y_offset
			else:
				polygons[tri_ind][point_ind,1] = polygons[tri_ind][point_ind,1]-abs(min_y)+y_offset
	return polygons

surface_tris = reset_origin_tris(surface_tris,70,15)
print(len(surface_tris))
print(surface_tris)


min x  181.5282
min y  710.7177
1260
[[[147.36 160.54   4.  ]
  [147.04 151.02   4.  ]
  [148.51 156.65   4.  ]]

 [[148.51 156.65   4.  ]
  [147.04 151.02   4.  ]
  [149.42 152.24   4.  ]]

 [[149.42 152.24   4.  ]
  [147.04 151.02   4.  ]
  [146.97 133.13   4.  ]]

 ...

 [[128.25 130.61   4.  ]
  [128.04 121.64   4.  ]
  [128.52 119.72   4.  ]]

 [[128.25 130.61   4.  ]
  [128.52 119.72   4.  ]
  [129.24 117.99   4.  ]]

 [[127.83 127.06   4.  ]
  [127.79 123.77   4.  ]
  [128.04 121.64   4.  ]]]


In [58]:
# moving everything to the first quadrant.

def reset_origin_polys(polygons, x_offset=0.0, y_offset=0.0):
	min_x =  float("inf")
	min_y =  float("inf")
	# getting minimum indices
	for poly_ind in range(int(len(polygons))):
		# if(surface_tris[tri_ind].shape[0] != 3):
		# 	print("Shape is not a triangle! ", str(surface_tris[tri_ind].shape[0]), " Points detected!")
		# 	print(surface_tris[tri_ind])
		for point_ind in range(polygons[poly_ind].shape[0]):
			if polygons[poly_ind][point_ind,0] < min_x:
				min_x = polygons[poly_ind][point_ind,0] 
			if polygons[poly_ind][point_ind,1] < min_y:
				min_y = polygons[poly_ind][point_ind,1] 
	
	print()
	# updating all points
	for poly_ind in range(int(len(polygons))):

		
		for point_ind in range(polygons[poly_ind].shape[0]):
			if(min_x<0):
				polygons[poly_ind][point_ind,0] = polygons[poly_ind][point_ind,0]+abs(min_x)+x_offset
			else:
				polygons[poly_ind][point_ind,0] = polygons[poly_ind][point_ind,0]-abs(min_x)+x_offset

			if(min_y<0):
				polygons[poly_ind][point_ind,1] = polygons[poly_ind][point_ind,1]+abs(min_y)+y_offset
			else:
				polygons[poly_ind][point_ind,1] = polygons[poly_ind][point_ind,1]-abs(min_y)+y_offset


	return polygons

print(polygons)
polygons = reset_origin_polys(polygons,45,15)
print(polygons)



[array([[300.95, 847.96,   4.  ],
       [301.51, 843.12,   4.  ],
       [301.64, 838.31,   4.  ],
       [301.59, 833.32,   4.  ],
       [301.64, 829.31,   4.  ],
       [301.94, 825.29,   4.  ],
       [303.13, 818.43,   4.  ],
       [303.54, 815.88,   4.  ],
       [303.57, 813.76,   4.  ],
       [303.17, 812.3 ,   4.  ],
       [302.25, 811.13,   4.  ],
       [300.69, 810.16,   4.  ],
       [298.44, 809.41,   4.  ],
       [297.37, 809.19,   4.  ],
       [298.12, 819.23,   4.  ],
       [298.49, 828.85,   4.  ],
       [298.57, 846.74,   4.  ],
       [298.88, 856.26,   4.  ],
       [300.04, 852.37,   4.  ]], dtype=float32), array([[333.56, 761.49,   4.  ],
       [338.44, 762.12,   4.  ],
       [341.48, 762.42,   4.  ],
       [343.75, 762.43,   4.  ],
       [344.67, 762.27,   4.  ],
       [345.15, 762.08,   4.  ],
       [345.29, 761.98,   4.  ],
       [345.35, 761.84,   4.  ],
       [345.41, 761.38,   4.  ],
       [345.33, 760.51,   4.  ],
       [345.05, 759.48,  

In [59]:
# # exporting to a file
# print(np.shape(surface_tris))

# num_polys = len(polygons)
# output_name = "neuro_model.txt"

# with open(output_name, "wb") as f:
# 	f.write(str("num polygons\n").encode())
# 	f.write(str(str(num_polys) + "\n\n").encode())
# 	for poly_ind in range(int(num_polys)):
# 		f.write(str(polygons[poly_ind].shape[0]).encode())
# 		f.write(("\n").encode())
# 		for point_ind in range(polygons[poly_ind].shape[0]):
# 			# f.write(str(str(surface_tris[tri_ind,point_ind, 0]), " ", str(surface_tris[tri_ind,point_ind, 1])).encode())
# 			np.savetxt(f,[polygons[poly_ind][point_ind,0:2]], fmt='%.2f',delimiter=',')
# 		f.write(b"\n")

# exporting to a file
print(np.shape(surface_tris))

num_tris = len(surface_tris)
output_name = "neuro_model_tris.txt"

with open(output_name, "wb") as f:
	f.write(str("num polygons\n").encode())
	f.write(str(str(num_tris) + "\n\n").encode())
	for tri_ind in range(int(num_tris)):
		# print(tri_ind)
		f.write(str(surface_tris[tri_ind].shape[0]).encode())
		f.write(("\n").encode())
		for point_ind in range(surface_tris[tri_ind].shape[0]):
			np.savetxt(f,[surface_tris[tri_ind,point_ind, 0:2]], fmt='%.2f',delimiter=',')
			# f.write(str(str(surface_tris[tri_ind,point_ind, 0]), " ", str(surface_tris[tri_ind,point_ind, 1])).encode())
		f.write(b"\n")

(1260, 3, 3)
